# `.loc` and `.iloc` indexing

From the [What is a Series section](what-is-a-series), remember our maxim:

> A *Series* is the association of:
>
> * An array of values (`.values`)
> * A sequence of labels for each value (`.index`)
> * A name (which can be `None`).

On this page we will examine differences between Data Frame attributes. We
will demonstrate that the `name` attribute is optional. The default `name` is
`None` and the `name` attribute can be left unspecified in any given Series
with minimal consequences. We will show, however, that it is important to
specify the `name` attribute as a column name when a Series is part of
a Pandas Data Frame.

In contrast, an `index` is essential - if no `index` is specified then Pandas will create one. However, we will show that it is sensible to replace Pandas' default index with a custom index, to avoid accidental errors when indexing. 

In [ ]:
# import libraries
import numpy as np
import pandas as pd

## What's in a `name`?

To consider the difference in importance/default behaviour between the `name`
attribute and the `index` attribute, let's again build a Pandas Series from
component parts, using Numpy arrays. We'll use the [fertility and Human
Development Index data once more](data/data_notes).

First, an array containing the three-letter codes for each country:

In [ ]:
# three letter codes for each country
country_codes_array = np.array(['AUS', 'BRA', 'CAN',
                                'CHN', 'DEU', 'ESP',
                                'FRA', 'GBR', 'IND',
                                'ITA', 'JPN', 'KOR',
                                'MEX', 'RUS', 'USA'])
country_codes_array

Second, an array containing the Human Development Index (HDI) scores for each country:

In [ ]:
# Human Development Index Scores for each country
hdis_array = np.array([0.896, 0.668, 0.89,
                       0.586, 0.89,  0.828,
                       0.844, 0.863, 0.49,
                       0.842, 0.883, 0.824,
                       0.709, 0.733, 0.894])
hdis_array

We can make a Series using the `pd.Series()` constructor. Our `.values` array is the HDI scores, and the `index` is the three-letter country codes. As we have seen previously, we specify the `index` using the `index=` argument.

In [ ]:
# make a series from the `hdis` array
hdi_series =  pd.Series(hdis_array, 
                        index=country_codes_array)
hdi_series

Currently, our Series has data (`values`) and an `index`, but no `name` attribute:

In [ ]:
# show the `values`
hdi_series.values

In [ ]:
# show the `index`
hdi_series.index

In [ ]:
# show that `name` is None (e.g. does not exist - this cell will produce no output)
hdi_series.name

In [ ]:
# show that `name` is None (e.g. does not exist)
hdi_series.name is None

The `name` attribute is optional when considering Series which are not part of Data Frames. If we do not specify a `name`, Pandas will set `name` to be `None`.

However, the name attribute can be useful for tracking what data is inside the `values` array. 

Let's make a new Series - called `hdi_series_with_name_and_index` - where we **do** specify a `name` attribute when calling the `pd.Series()` constructor. 

To specify the `name` attribute we use the `name = ` argument inside the `pd.DataFrame()` constructor:

In [ ]:
# Make a series from the `hdis` array, specifying the `name` attribute.
hdi_series_with_name_and_index = pd.Series(hdis_array,
                                           index=country_codes_array,
                                           name = 'Human Development Index')
hdi_series_with_name_and_index

Now, `name` is no longer equal to `None`:

In [ ]:
# Show the `name` attribute
hdi_series_with_name_and_index.name

In [ ]:
# `name` is not None
hdi_series_with_name_and_index.name is None

## What's in an `index`?

So far so good, we have seen what happens if we make a Series with no `name` attribute. But what if we make a Series without specifying an `index`? 

To do this, we simply call the `pd.Series()` constructor without using the `index = ` argument:

In [ ]:
# make a series from the `hdis` array, with no `name` and no `index` specified
hdi_series_no_name_no_index =  pd.Series(hdis_array)
hdi_series_no_name_no_index

Here, you can see that while we did not specify an `index` Pandas has automatically generated an one:

In [ ]:
# view the new Series
hdi_series_no_name_no_index

Let's take a closer look at this `index` by directly accessing the `.index`
attribute:

In [ ]:
# The default Pandas index
hdi_series_no_name_no_index.index

Hmm, it is quite strange-looking relative to the index of country codes (which
visually resembles a numpy array). Let's use to the `type()` function to
investigate further:

In [ ]:
# so the type of the default index
type(hdi_series_no_name_no_index.index)

Ok, so for this Series - where we did not specify an `index` - the index that Pandas has created is a `RangeIndex` object. This is, in fact, the default Pandas index that will be attached to a Series where no other index is specified.

The `RangeIndex` looks like a sequence of numbers (and in a sense it is) - but it behaves differently to python list and arrays with regards to indexing.

Let's take a look at the shape of the `RangeIndex` and what is in it. To view the shape, we can use the `np.shape()` function:

In [ ]:
# take a closer look at the "default" index
np.shape(hdi_series_no_name_no_index.index)

Ok, so the index has 15 elements. Let's view the first one, using direct indexing:

In [ ]:
# view the first element of the RangeIndex
hdi_series_no_name_no_index.index[0]

Again, so far, so good. Now let's view the second element:

In [ ]:
# view the second element of the RangeIndex
hdi_series_no_name_no_index.index[1]

Previously, we used three-letter country codes as the index elements (these were strings).

Let's have a look at the `type()` of the numbers in the `RangeIndex`:

In [ ]:
# what type are the elements of the range index?
type(hdi_series_no_name_no_index.index[0])

You might be fooled here into thinking that the `RangeIndex` behaves like a python list. If we try to use slicing, we will see this is not the case:

In [ ]:
# slice the RangeIndex
hdi_series_no_name_no_index.index[0:-1]

In [ ]:
# slice the RangeIndex
hdi_series_no_name_no_index.index[0:4]

This might seem like pretty weird behaviour. This is because the `RangeIndex` behaves similarly to the in-built python `range()` function, rather than as a list or numpy array (e.g. we cannot slice it using the familiar `start:stop` syntax).

To view all the values inside the `RangeIndex` we can loop over it:

In [ ]:
# loop over the index
for el in hdi_series_no_name_no_index.index:
    print(el)

Or show the values together by converting the index to a python list, using the `list()` function:

In [ ]:
# show the elements in the RangeIndex using list()
list(hdi_series_no_name_no_index.index)

## Why the "default" index can be confusing

To recap: we've used three-letter country codes as the elements of an `index`, and we've just seen what happens if we construct a Data Frame without telling Pandas what to use as an `index` - it will create a default `RangeIndex`.

What is the advantage of using a non-default index? Below are some potential pitfalls to be aware of when using the default index.

Let's say we want to access the fifth element of the Series. This is at integer location 4, because we count from 0. At the moment the numerical labels from `RangeIndex` "line up" with the integer-based locations:

In [ ]:
# show the whole Series
hdi_series_no_name_no_index

If we use integer indexing (`.iloc`) we get the same value as if we use label based indexing (`.loc`):

In [ ]:
# indexing using integer location
hdi_series_no_name_no_index.iloc[4]

In [ ]:
 # indexing using labels (from the default index)
hdi_series_no_name_no_index.loc[4]

What if we don't tell pandas what type of indexing we want to do? E.g. we do not use `.iloc` or `.loc`, we just use the sort of direct indexing we would use on a python list?

In [ ]:
# direct indexing
hdi_series_no_name_no_index[4]

All of these methods, presently, have returned the same value.

**But  this will not always be the case.** Certain functions and methods that we may want to use to sort and organise our data will cause cause misalignment between the `RangeIndex` and the integer location of a given element of the Series.

For instance let's sort the data in our `hdi_series_no_name_no_index` Series in ascending order.  To do this we will use the `.sort_values()` method. We will cover Pandas methods in detail on [later pages](0_2_pandas_dataframes_attributes_methods.Rmd)  but for now the `.sort_values()` method just as what it says on the tin it sorts the elements of the Series in ascending order, based on the elements in the `.values` attribute of the Series:

In [ ]:
# sorting the values in ascending order
hdi_series_no_name_no_index_sorted = hdi_series_no_name_no_index.sort_values()

hdi_series_no_name_no_index_sorted

Look at the left hand side of the print out from the cell above e.g. look at the `RangeIndex`.  The numbers within the `RangeIndex` no longer run sequentially from 0 to 14. This means that the integer location of each element in the Series no longer matches up with the index label. This can potentially be a source of errors.

Let's see what happens if we try to access the fifth element of the series using integer based indexing (`.iloc[4]`) location based indexing (`.loc[4]`) and direct indexing (`[4]`) as we did above.

When we did this on the unsorted data all these methods returned the same value:

In [ ]:
# integer indexing on the sorted data
hdi_series_no_name_no_index_sorted.iloc[4]

In [ ]:
# label indexing on the sorted data
hdi_series_no_name_no_index_sorted.loc[4]

In [ ]:
# direct indexing on the sorted data
hdi_series_no_name_no_index_sorted[4]

Oh dear,  we have used the number 4 with each indexing method, yet have gotten back different values. 

This is a pitfall of using the default `RangeIndex` - it can lead to confusing results when the integer-based location and the `int` label of an element of the Series do not match up. 

Compare this to our `hdi_series_with_name_and_index` which uses the three-letter country codes as it's index:

In [ ]:
# show the `hdi_series_with_name_and_index` Series
hdi_series_with_name_and_index

Let's get the Fifth Element using integer based (`.iloc`) indexing:

In [ ]:
# integer indexing
hdi_series_with_name_and_index.iloc[4]

...and let's try to use `.loc[4]` on this Series (this will generate an error):

In [ ]:
# label indexing raises a KeyError ...
hdi_series_with_name_and_index.loc[4]

This `KeyError` tells us that there is no index label `4` (which makes sense as the index labels in this Series are three-letter country codes). To use `.loc` with this Series, we must use the three-letter country code strings:

In [ ]:
# label based indexing
hdi_series_with_name_and_index.loc['DEU']

Direct indexing here will assume we mean `.iloc` integer-based indexing.

In [ ]:
# direct indexing
hdi_series_with_name_and_index[4]

Using a custom index (e.g. the three-letter country codes) rather than the default `RangeIndex` has the advantage of avoiding potential confusion between the integer location of a datapoint, and the index label of that datapoint.

To demonstrate this, let's sort our `hdi_series_with_name_and_index` in ascending order:

In [ ]:
# sorting the Series in ascending order
hdi_series_with_name_and_index_sorted = hdi_series_with_name_and_index.sort_values()

hdi_series_with_name_and_index_sorted

The use of custom string-based labels in the index (e.g. `FRA`, `AUS` etc)  and say confusing misalignment between `RangeIndex` numerical labels and integer location.

Now, if we index using a number, this means *integer indexing*:

In [ ]:
# index using `.iloc`
hdi_series_with_name_and_index_sorted.iloc[4]

In [ ]:
# index using direct indexing.
# Currently this generates a warning.  In the future it will be an error.
hdi_series_with_name_and_index_sorted[4]

Using `.loc` means we have to use a string, preventing errors where we use a number and return data we do not expect.

In [ ]:
# label-based indexing
hdi_series_with_name_and_index_sorted.loc['DEU']

However, a good additional maxim is: **use `.loc` and `iloc` unless there is a good reason not to!**.

If you use direct indexing, e.g. `[4]` without using `.iloc` or `.loc`, then Pandas assumes you want to use integer indexing.  if this is the case it is better to be explicit and use the `.iloc` method *especially if the Series has a `RangeIndex` with numeric labels*.

## Strange and `name`less Data Frame columns

When we include Series as columns in Data Frames - remember, that a Data Frame is just a dictionary-like collection of Series - it becomes eminently sensible to specify a `name` attribute for each Series.

Pandas will not *force* us to do this, but it leads to some error-prone consequences if we do not. In fact, the `RangeIndex` crops up again here, and can create confusion in similar ways to the ones we have seen in the last section.

As shown above, our `hdi_series` does not have a `name` attribute. Let's pass this Series to the `pd.DataFrame()` constructor, to see the consequence of a `name`less Series in this context. We will call the resulting Data Frame `no_name_df`:

In [ ]:
# verify again that the `hdi_series` has no `name` attribute
hdi_series.name is None

In [ ]:
# a Data Frame made of a Series with no `name` attribute
no_name_df = pd.DataFrame(hdi_series)

no_name_df

Ok, so in the absence of a `name` attribute Pandas has labelled the column with a `0`. If we inspect more deeply, we find that actually Pandas, in the absence of being instructed otherwise, has created a `RangeIndex`, but this time for the `columns` (e.g. the column names) of the Data Frame. If you look at the Data Frame above, you can see that the `index` attribute is the three-letter country codes. However, the `.columns` are a `RangeIndex`:

In [ ]:
# look at the column names via the `.columns` attribute
no_name_df.columns

Sure enough, if we check the `type` of the fist element in this `RangeIndex` it is a of a numeric type - e.g. we can think of it not as a column *name* but a number standing in for a column name:

In [ ]:
# check the type of the first element in the `.columns` attribute
type(no_name_df.columns[0])

It may be obvious why this naming convention for Data Frame columns can lead to errors for reasons of low interpretability. We typically want our column names to be descriptive of the data in the column, to ourselves and to other people reading our code. If we do not specify a `name` attribute for our Series when creating Data Frames, the default numerical column names supplied by Pandas are hard to interpret, and it is easy to misinterpret, or to forget what data is in that column, leading to human errors.

They can also lead to indexing errors, similar to those we saw in the previous section. To demonstrate this, let's compare the `name`less Data Frame above to a Data Frame created from a Series with a `name`.

Our aptly named `hdi_series_with_name_and_index` has a `name` attribute - (look at the last line of the output from the cell below, the `Name: Human Development Index`):

In [ ]:
# a Series with a `name`
hdi_series_with_name_and_index

Let's call the `pd.DataFrame()` constructor on this Series - we'll call the resultant Data Frame `named_df`:

In [ ]:
# create a new Data Frame, with a sensible name for the column
named_df = pd.DataFrame(hdi_series_with_name_and_index)

named_df

We see that Pandas has automatically used the `name` attribute as the column name, hugely increasing the interpretability of the resulting Data Frame.

We can see the `name` in the `.columns` attribute of the new  `named_df` Data Frame:

In [ ]:
# show the column names from the `named_df` Data Frame
named_df.columns

Now, let's try using direct indexing with each Data Frame, using the column names.

This is very straightforward for the `named_df`:

In [ ]:
# direct indexing to retrieve a column by name
named_df['Human Development Index']

What about for `no_name_df`? Well, the column name there is `0`, of `int` type.

What happens if use direct indexing? We are hoping we see something like the output of the cell above, albeit with a `0` for the `name`, rather than `Human Development Index`:

In [ ]:
# direct indexing with no `name` attribute
no_name_df[0]

Sure enough that is what we see. But the operation we have used looks very much like *integer* indexing on a Series, which will return a single value:

In [ ]:
# integer indexing a Series (without using `.iloc`)
hdi_series[0]

This can be confusing.

In addition, the numerical label can also be confusing if we introduce more columns, especially if we mix in columns which do have `name` attributes that we specify.

For instance, if we add in the full name of each country:

In [ ]:
# country names array
country_names_array = np.array(['Australia', 'Brazil', 'Canada',
                          'China', 'Germany', 'Spain',
                          'France', 'United Kingdom', 'India',
                          'Italy', 'Japan', 'South Korea',
                          'Mexico', 'Russia', 'United States'])
country_names_array

Let's add this into the `no_name_df`, using the `name` "Country Names":

In [ ]:
no_name_df['Country Names'] = country_names_array

no_name_df

We now have one column that has an `int` as it's column name, and another with a string.

In the same way as for a numerical `index`, this situation can become confusing if the numerical labels become misaligned with their integer location in the `.columns` attribute.

For instance, if we re-arrange the columns:

In [ ]:
# re-arrange the columns
no_name_df = no_name_df[["Country Names", 0]]

no_name_df

We now have a column with the `name` "0" that is not at the 0-th location of the `.columns` attribute:

In [ ]:
# show the 0-th element of the `.columns` attribute
no_name_df.columns[0]

We are now in the precarious situation of using a `int` `0` both as a *label* and as a *location*. In the cell above, it is a location, it the cell below it serves as a label:

In [ ]:
# direct index the `0` column
no_name_df[0]

Ideally, we want a clear separation between integer indexes (like `0`) and *column names*. 

This confusing situation can be completely avoided in the case of `named_df`. Let's add the country names to this Data Frame also:

In [ ]:
# add the country names
named_df['Country Names'] = country_names_array

named_df

We don't introduce any confusion or error-proneness by re-arranging these columns which have sensible string names:

In [ ]:
# re-arrange the columns
named_df = named_df[["Country Names", "Human Development Index"]]

named_df

In fact, now if we want to use integer indexing, we are forced to use `.iloc`, as else we get an error:

In [ ]:
# this will not work if each column has a `name` string
named_df[0]

This compels us to stick to the good practice maxim we introduced above: **use `.loc` and `iloc` unless there is a good reason not to!**. 

Giving every column a sensible string as a `name` is one situation where direct indexing (e.g. in the present context using `named_df['Human Development Index']` etc.) is safe and non-error prone. As we have seen, issues can arise if we do not specify `name` attributes, and let Pandas automatically generate numeric labels for our Data Frame columns...

## Summary

On this page we have looked at the differences between attributes of the Pandas Series - the `name` is optional, but a default `RangeIndex` will be supplied if no custom `index` is specified.

We have seen that the `RangeIndex` can lead to errors if the numeric row labels become misaligned with the integer location of a given row. Similarly, if we do not specify a `name` attribute for Data Frame columns, Pandas will generate numeric labels which can create confusion between integer-based and direct indexing.

For best results, we should specify both an interpretable `index` and interpretable `name` attributes for our Series, especially when they are part of Data Frames.